# Unit 2: Popularity Recommendations

In this section we build a recommender that sorts items by popularity as of the number of ratings they received. As a result we return the $N$ most popular items as recommendations.

In [102]:
%load_ext autoreload
%autoreload 2

from typing import Dict, List
import os
import sys
import math

import numpy as np
import scipy as sp
import sklearn

import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [3]:
ml100k_ratings_filepath = '../data/raw/ml-100k/u.data'

## Load Data

In [8]:
data = Dataset(ml100k_ratings_filepath)
data.rating_split(seed=42)

## Popularity Ranking

How do we define _popularity_? It turns out that there can be different things justifying the popularity of content:
- pure count: simply count the number of ratings or interactions an item received regardless of their quality
- positive count: only count the number of ratings or interactions that we assume reflect preference towards items, e.g. ratings above user mean ratings
- time-dependency: despite evergreen stars items may also be popular for a limited time only - how can we account for this?

However, popularity ranking entails no personalization. We obtain a single popularity ranking of items which is independent from the user and serve the same top-$k$ items to every user.

### Popularity based on simple Interaction Counts

In [13]:
item_popularity = data.train_ratings['item'].value_counts()

In [14]:
item_popularity

50      471
181     423
258     409
100     405
294     394
       ... 
1320      1
1669      1
1576      1
1541      1
1663      1
Name: item, Length: 1651, dtype: int64

In [68]:
item_order = item_popularity.index.values

### Popularity based on positive Interaction Counts

Therefore we must first remove all ratings that fall below the user rating.

check that there are less hits, but on average the hits have a higher rating and thus are more relevant - we are recommending less, but also less shit

check spearman rank correlation between both orderings to quantify the distortion in ordering

In [42]:
user_mean_ratings = data.train_ratings[['user', 'rating']].groupby('user').mean().reset_index()
user_mean_ratings.rename(columns={'rating': 'user_mean_rating'}, inplace=True)

In [43]:
user_mean_ratings

,user,user_mean_rating
0,1,3.590476
1,2,3.673469
2,3,2.809524
3,4,4.333333
4,5,2.925373
...,...,...
938,939,4.292683
939,940,3.425287
940,941,3.947368
941,942,4.234375


In [59]:
positive_train_ratings = data.train_ratings.merge(user_mean_ratings, on='user', how='left')

In [60]:
keep_ratings = positive_train_ratings['rating'] >= positive_train_ratings['user_mean_rating']

In [62]:
positive_train_ratings = positive_train_ratings[keep_ratings]
positive_train_ratings.drop(columns='user_mean_rating', inplace=True)

/anaconda3/envs/recsys_training/lib/python3.7/site-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


In [63]:
positive_train_ratings

,user,item,rating,timestamp
0,877,381,4,882677345
2,94,431,4,891721716
6,598,286,5,886711452
7,886,496,4,876031952
9,521,184,4,884478358
...,...,...,...,...
79989,200,673,5,884128554
79993,44,432,5,878347569
79995,92,48,4,875653307
79997,1,96,5,875072716


In [65]:
item_popularity_positive = positive_train_ratings.item.value_counts()

In [67]:
item_popularity_positive

50      395
100     319
181     303
174     278
127     273
       ... 
1534      1
1598      1
1630      1
1662      1
1631      1
Name: item, Length: 1451, dtype: int64

In [70]:
item_order_positive = item_popularity.index.values

#### How strong do both orderings correlate with each other?

In [90]:
joint_counts = [[item_popularity.loc[item], item_popularity_positive[item]]
                for item in np.intersect1d(item_popularity_positive.index.values, item_popularity.index.values)]
joint_counts = np.array(joint_counts)

In [91]:
joint_counts

array([[365, 252],
       [105,  40],
       [ 72,  28],
       ...,
       [  4,   3],
       [  1,   1],
       [  1,   1]])

In [92]:
sp.stats.spearmanr(joint_counts)

SpearmanrResult(correlation=0.946241194276375, pvalue=0.0)

### Using Popularity Ordering for top-$N$ Recommendations

In [93]:
item_order

array([  50,  181,  258, ..., 1576, 1541, 1663])

In [94]:
item_order_positive

array([  50,  181,  258, ..., 1576, 1541, 1663])

In [98]:
N = 10

In [113]:
def get_recommendations(N) -> Dict[int, None]:
    recs = dict(zip(item_order[:N], [None]*N))
    return recs

In [100]:
get_recommendations()

{50: None,
 181: None,
 258: None,
 100: None,
 294: None,
 286: None,
 288: None,
 1: None,
 121: None,
 300: None}

## Evaluation the Relevance of Recommendations

In [107]:
def get_relevant_items(test_ratings: pd.DataFrame) -> Dict[int, List[int]]:
    """
    returns {user: [items]} as a list of relevant items per user
    for all users found in the test dataset
    """
    relevant_items = test_ratings[['user', 'item']]
    relevant_items = relevant_items.groupby('user')
    relevant_items = {user: relevant_items.get_group(user)['item'].values
                      for user in relevant_items.groups.keys()}

    return relevant_items

In [104]:
relevant_items = get_relevant_items(data.test_ratings)

In [106]:
relevant_items[1]

array([143,  88, 131,  15, 239,  99, 255, 134, 210, 264, 227, 202,  59,
        17, 203,   9,  39,  55, 251, 213,  27, 137, 180, 207, 196, 101,
        28, 123,  13, 268, 106, 112, 222,  93, 126, 246, 257,  98,  34,
       130, 218, 188, 242, 270, 170, 122, 120, 224,  72, 183, 190, 146,
       167, 214, 121,   2,   5, 182,  43, 175, 252, 184])

$Precision@10$

Now, we can compute the intersection between the top-$N$ recommended items and the items each user interacted with. Ideally, we want every recommendation to be a hit, i.e. an item the user consumed. In this case the size of intersections is $N$ given $N$ recommendations which is a precision of 100% = $\frac{N}{N}$.

We compute the so called $Precision@N$ for every user and take the mean over all. The resulting metric is called _mean average precision at N_ or short $MAP@N$.

Task: Compute the MAP@N for popularity recommendations

#### Item Order

In [133]:
users = relevant_items.keys()
prec_at_N = dict.fromkeys(users)
recommendations = item_order[:N]

for user in users:
    hits = np.intersect1d(recommendations, relevant_items[user])
    prec_at_N[user] = len(hits)/N

In [126]:
np.mean(list(prec_at_N.values()))

0.10085106382978724